# Пользовательские логи

В файле log.csv хранятся логи пользовательской активности. Каждая строка в файле — это одно событие, которое описано в формате: время, идентификатор пользователя, тип события, параметр события.

На основе файла log.csv выполните следующие задания:

Назовем сессией последовательный набор событий одного пользователя. Сессия заканчивается, если пользователь не совершал действий в течение 30 минут или более. Склейте события в сессии. В ответе укажите число сессий, которые начались 2020-04-14.
Будем считать, что пользователь слушал музыку, если у него было событие с типом 1 и параметром music. Найдите день, когда число уникальных пользователей, слушавших музыку, было максимальным. В ответе укажите этот максимум.
Найдите пятнадцатиминутный интервал [time; time + 15 minutes), в течение которого произошло больше всего событий. В ответе укажите время начала интервала в формате YYYY-MM-DD_hh:mm:ss. Если таких интервалов несколько, укажите начало наиболее позднего интервала. Обратите внимание, что началом интервала может быть произвольный момент времени.

In [2]:
import pandas as pd

In [3]:
import datetime as dt

In [11]:
df = pd.read_csv('log.csv')

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        200000 non-null  object        
 1   user        200000 non-null  int64         
 2   event_type  200000 non-null  int64         
 3   parameter   160043 non-null  object        
 4   datetime    200000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 7.6+ MB


In [5]:
df.head()

,date,user,event_type,parameter
0,2020-04-01_00:05:46,180397,3,NaN
1,2020-04-01_00:00:21,662939,4,music
2,2020-04-01_00:20:50,703220,2,NaN
3,2020-04-01_00:22:31,405034,1,unknown
4,2020-04-01_00:30:21,662939,0,text


проверяю работу datetime

Обход датафрейма. Но это занимает очень много ресурсов и времени. Неэффективно

Попробую использовать метод apply

In [12]:
df['datetime'] = df.apply(lambda row: dt.datetime.strptime(row['date'], '%Y-%m-%d_%H:%M:%S'), axis=1)

Получилось гораздо быстрее. Продолжаем

Сортируем датафрейм по дате, обновляем индексы

In [13]:
df = df.sort_values(by='datetime').reset_index(drop='True')

In [8]:
df.head()

,date,user,event_type,parameter,datetime
0,2020-04-01_00:00:21,662939,4,music,2020-04-01 00:00:21
1,2020-04-01_00:05:46,180397,3,NaN,2020-04-01 00:05:46
2,2020-04-01_00:20:50,703220,2,NaN,2020-04-01 00:20:50
3,2020-04-01_00:22:31,405034,1,unknown,2020-04-01 00:22:31
4,2020-04-01_00:30:21,662939,0,text,2020-04-01 00:30:21


Создаем ses - список сессий. У каждой сессии три параметра: дата и время начала, дата и время последнего действия, номер пользователя

Создаем первую запись из первой строки датафрейма

In [70]:
ses = []
ses += [[df.loc[0,'datetime'], df.loc[0,'datetime'], df.loc[0,'user']]]
#print(ses)
#print(ses[0][1])

Проверяла работу на маленьком датафрейме

In [71]:
df2 = df.head(10000)

Запускаем на полном датафрейме

In [72]:
for row in df2.itertuples():
    n = 0
    #print('break2')
    #print(row)
    #print(row[5])
    #print(ses[-1])
    for i in ses:
        #print('break1')
        #print(row[2])
        #print(i[2])
        #print(i[1])
        if row[2] == i[2] and row[5] == i[1]:
            n = 1
        elif row[2] == i[2] and (row[5] - i[1]) < dt.timedelta(0, 30):
            i[1] = row[5]
            n = 1
    if n == 0:
        ses += [[row[5], row[5] , row[2]]]   
#del ses[0]            

In [73]:
len(ses)

10000

In [41]:
for i in ses:
    print(i)

[Timestamp('2020-04-01 00:00:21'), Timestamp('2020-04-01 00:00:21'), 662939]
[Timestamp('2020-04-01 00:05:46'), Timestamp('2020-04-01 00:05:46'), 180397]
[Timestamp('2020-04-01 00:20:50'), Timestamp('2020-04-01 00:20:50'), 703220]
[Timestamp('2020-04-01 00:22:31'), Timestamp('2020-04-01 00:22:31'), 405034]
[Timestamp('2020-04-01 00:30:21'), Timestamp('2020-04-01 00:30:21'), 662939]
[Timestamp('2020-04-01 00:31:00'), Timestamp('2020-04-01 00:31:00'), 943401]
[Timestamp('2020-04-01 00:33:28'), Timestamp('2020-04-01 00:33:28'), 283417]
[Timestamp('2020-04-01 00:35:46'), Timestamp('2020-04-01 00:35:46'), 180397]
[Timestamp('2020-04-01 00:36:43'), Timestamp('2020-04-01 00:36:43'), 375105]
[Timestamp('2020-04-01 00:39:13'), Timestamp('2020-04-01 00:39:13'), 231944]


In [64]:
for i in ses:
    #print(i)
    if i[2] == 529241:
        print(i)

[Timestamp('2020-04-01 02:15:06'), Timestamp('2020-04-01 02:15:06'), 529241]
